In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from category_encoders import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from shapash.explainer.smart_explainer import SmartExplainer

url = "https://raw.githubusercontent.com/remijul/dataset/master/Airline%20Passenger%20Satisfaction.csv"
data = pd.read_csv(url, delimiter=';')

data.dropna(inplace=True)
categorical_columns = ['Gender', 'Customer Type', 'Type of Travel', 'Class']
data[categorical_columns] = data[categorical_columns].astype('category')
data['Satisfaction'] = data['Satisfaction'].map({'satisfied': 1, 'neutral or dissatisfied': 0})

X = data.drop('Satisfaction', axis=1)
y = data['Satisfaction']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

numerical_columns = ['Age', 'Flight Distance', 'Seat comfort', 'Departure/Arrival time convenient', 'Food and drink', 'Gate location', 'Inflight wifi service', 'Inflight entertainment', 'Online support', 'Ease of Online booking', 'On-board service', 'Leg room service', 'Baggage handling', 'Checkin service', 'Cleanliness', 'Online boarding', 'Departure Delay in Minutes', 'Arrival Delay in Minutes']

numerical_transformer = StandardScaler()
categorical_transformer = OrdinalEncoder()

preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_transformer, numerical_columns),
    ('cat', categorical_transformer, categorical_columns)
])

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')
print(classification_report(y_test, y_pred))

Accuracy: 0.96
              precision    recall  f1-score   support

           0       0.95      0.96      0.96     11821
           1       0.97      0.96      0.96     14077

    accuracy                           0.96     25898
   macro avg       0.96      0.96      0.96     25898
weighted avg       0.96      0.96      0.96     25898



In [22]:
# Encoder les données de test avec le preprocessor
X_test_enc = preprocessor.transform(X_test)

In [23]:
# Instancier SmartExplainer
xpl = SmartExplainer(
    model=pipeline.named_steps['classifier'],
    preprocessing=preprocessor
)

In [32]:
X_test_enc

array([[-0.16109613,  1.75786533,  0.832696  , ...,  2.        ,
         1.        ,  2.        ],
       [ 1.35969628, -0.1925373 ,  0.11428522, ...,  2.        ,
         2.        ,  1.        ],
       [-0.82231023, -0.15257403, -0.60412557, ...,  2.        ,
         1.        ,  2.        ],
       ...,
       [-1.81413137,  0.40983593, -0.60412557, ...,  1.        ,
         1.        ,  1.        ],
       [ 0.30175373, -1.58052997, -1.32253635, ...,  2.        ,
         2.        ,  1.        ],
       [ 0.83072501, -0.59314473, -0.60412557, ...,  2.        ,
         2.        ,  1.        ]])

In [34]:
# Compiler le SmartExplainer avec les données pertinentes
xpl.compile(
    x=pd.DataFrame(X_test_enc, columns=X_test.columns)
)

In [ ]:
app = xpl.run_app(title_story="Titre de l'application")

c:\Users\loren\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning:

X has feature names, but RandomForestClassifier was fitted without feature names

c:\Users\loren\anaconda3\lib\site-packages\shapash\webapp\smart_app.py:354: FutureWarning:

Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.

INFO:root:Your Shapash application run on http://Optimus:8050/
INFO:root:Use the method .kill() to down your app.


Dash is running on http://0.0.0.0:8050/



INFO:dash.dash:Dash is running on http://0.0.0.0:8050/



 * Serving Flask app "shapash.webapp.smart_app" (lazy loading)


 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://192.168.1.18:8050/ (Press CTRL+C to quit)
INFO:werkzeug:192.168.1.18 - - [07/Apr/2023 15:10:24] "GET / HTTP/1.1" 200 -
INFO:werkzeug:192.168.1.18 - - [07/Apr/2023 15:10:24] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_9_2m1680853277.12.1.min.js HTTP/1.1" 200 -
INFO:werkzeug:192.168.1.18 - - [07/Apr/2023 15:10:24] "GET /assets/material-icons.css?m=1680853278.2582488 HTTP/1.1" 200 -
INFO:werkzeug:192.168.1.18 - - [07/Apr/2023 15:10:24] "GET /assets/style.css?m=1680853278.262019 HTTP/1.1" 200 -
INFO:werkzeug:192.168.1.18 - - [07/Apr/2023 15:10:24] "GET /_dash-component-suites/dash/deps/react@16.v2_9_2m1680853277.14.0.min.js HTTP/1.1" 200 -
INFO:werkzeug:192.168.1.18 - - [07/Apr/2023 15:10:24] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_9_2m1680853277.14.0.min.js HTTP/1.1" 200 -
INFO:werkzeug:192.168.1.18 - - [07/Apr/2023 15:10:24] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_9_2m1680853277.8.1.min.js HTTP/1.1" 200 -
INFO:wer